## Distributed Data Parallel DeBERTa Training with HuggingFace Transformers on SageMaker

Amazon SageMaker's distributed library can be used to train deep learning models faster and cheaper. The data parallel feature in this library (smdistributed.dataparallel) is a distributed data parallel training framework that can work with a variety of frameworks including PyTorch and TensorFlow, as well as higher level toolkits such as HuggingFace transformers.

In July 2021, AWS and HuggingFace collaborated to introduce [HuggingFace Deep Learning Containers (DLCs)](https://huggingface.co/transformers/v4.8.2/sagemaker.html#deep-learning-container-dlc-overview) which have fully integrated support with SageMaker to make it easier to develop and ship cutting-edge NLP models.

In this notebook, we shall see how to leverage HuggingFace DLCs and the [SageMaker HuggingFace estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html) to pre-train [DeBERTa model](https://arxiv.org/abs/2006.03654) on [SQuAD dataset](https://rajpurkar.github.io/SQuAD-explorer/) for a question-answering task.

The outline of steps is as follows:

- Install pre-requisites required for using SageMaker and HuggingFace transformers.
- Pre-process the SQuAD dataset and stage it in Amazon S3.
- Use HuggingFace Estimator to pre-train DeBERTa model on SQuAD dataset.

NOTE: This example requires SageMaker Python SDK v2.X.

### Prepare SageMaker Environment

To get started, we need to set up the environment with a few pre-requisite steps.

#### Pre-requisite Installations


In [16]:
!pip install sagemaker botocore boto3 awscli --upgrade
!pip install transformers datasets --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 517 kB 9.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 8.6 MB 95.2 MB/s            
     |████████████████████████████████| 132 kB 122.0 MB/s            
     |████████████████████████████████| 3.8 MB 73.5 MB/s            
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl (636 kB)
  Using cached docutils-0.15.2-py3-none-any.whl (547 kB)
  Using cached colorama-0.4.3-py2.py3-none-any.whl (15 kB)
  Created wheel for sagemaker: f

  Using cached tokenizers-0.11.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.5 MB)
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
  Using cached tqdm-4.63.0-py2.py3-none-any.whl (76 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/PyYAML-6.0.dist-info/METADATA'



In [1]:
import botocore
import boto3
import sagemaker
import transformers
import datasets

print(f"sagemaker: {sagemaker.__version__}")
print(f"transformers: {transformers.__version__}")
print(f"datasets: {datasets.__version__}")

sagemaker: 2.68.0
transformers: 4.6.1
datasets: 1.6.2


Copy and run the following code if you need to upgrade ipywidgets for datasets library and restart kernel. This is only needed when prerpocessing is done in the notebook.

In [33]:
%%capture
import IPython
!conda install -c conda-forge ipywidgets -y
# has to restart kernel for the updates to be applied
IPython.Application.instance().kernel.do_shutdown(True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


#### SageMaker Environment

Note: If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for SageMaker. To learn more, see [SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html).

In [2]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it does not exists
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::570106654206:role/Dev
sagemaker bucket: sagemaker-us-west-2-570106654206
sagemaker session region: us-west-2


#### Preparing the SQuAD dataset

When using the 🤗 [Datasets library](https://github.com/huggingface/datasets), datasets can be downloaded directly with the following datasets.load_dataset() method:

from datasets import load_dataset
load_dataset('dataset_name')

If you'd like to try other training datasets later, you can simply use this method.

For this example notebook, we prepared the SQuAD v1.1 dataset in the public SageMaker sample file S3 bucket. The following code cells show how you can directly load the dataset and convert to a HuggingFace DatasetDict.

NOTE: The [SQuAD dataset](https://rajpurkar.github.io/SQuAD-explorer/) is under the [CC BY-SA 4.0 license terms](https://creativecommons.org/licenses/by-sa/4.0/).

In [21]:
import pandas as pd
import numpy as np
import json
from datasets import Dataset
from datasets import DatasetDict
from datasets.filesystems import S3FileSystem
pd.__version__

'1.3.4'

In [23]:
# helper function to grab the dataset and load into DatasetDict
import urllib.request
import json


def make_split(split):
    if split == "train":
        file = "https://sagemaker-sample-files.s3.amazonaws.com/datasets/text/squad/train-v1.1.json"
    elif split == "test":
        file = "https://sagemaker-sample-files.s3.amazonaws.com/datasets/text/squad/dev-v1.1.json"
    with urllib.request.urlopen(file) as f:
        squad = json.load(f)
        data = []
        for article in squad["data"]:
            title = article.get("title", "")
            for paragraph in article["paragraphs"]:
                context = paragraph["context"]  # do not strip leading blank spaces GH-2585
                for qa in paragraph["qas"]:
                    answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                    answers = [answer["text"] for answer in qa["answers"]]
                    # Features currently used are "context", "question", and "answers".
                    # Others are extracted here for the ease of future expansions.
                    if split == "train":
                        for i in range (1,100):
                            data.append(
                            {
                                "title": title,
                                "context": context,
                                "question": qa["question"],
                                "id": qa["id"],
                                "answers": {
                                    "answer_start": answer_starts,
                                    "text": answers,
                                },
                            }
                            )
                    else:
                        data.append(
                            {
                                "title": title,
                                "context": context,
                                "question": qa["question"],
                                "id": qa["id"],
                                "answers": {
                                    "answer_start": answer_starts,
                                    "text": answers,
                                },
                            }
                            )
        df = pd.DataFrame(data)
        return Dataset.from_pandas(df)


train = make_split("train")
test = make_split("test")

datasets = DatasetDict()
datasets["train"] = train
datasets["validation"] = test
datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers'],
        num_rows: 8672301
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers'],
        num_rows: 10570
    })
})

#### Pre-processing the SQuAD dataset

Before we can feed those texts to the Trainer model, we need to preprocess them. This can be done by a 🤗 Transformers Tokenizer which (as the name indicates) tokenizes the input texts (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put them into a format the model expects, as well as generate other inputs that the model requires.

To do this, we instantiate a tokenizer using the AutoTokenizer.from_pretrained method, which will ensure that:

- We get a tokenizer that corresponds to the model architecture we want to use.
- We download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again when you run the cell.

In [24]:
model_checkpoint = "microsoft/deberta-base"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing. You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/docs/transformers/index#bigtable).

In [25]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = (
    128  # The authorized overlap between two parts of the context when splitting it is needed.
)
pad_on_right = tokenizer.padding_side == "right"

Now, let's put everything together in one function that we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the `cls` index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Because the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [26]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possibly giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation, and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [27]:
tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
)

In [28]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

train_dataset.set_format(
    "torch", columns=["attention_mask", "end_positions", "input_ids", "start_positions"]
)
eval_dataset.set_format(
    "torch", columns=["attention_mask", "end_positions", "input_ids", "start_positions"]
)

Before we kick off our SageMaker training job we need to transfer our dataset to S3 so the training job can download it from S3.

In [29]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()

s3_prefix = "samples/datasets/squad-8million"

# save train_dataset to s3
training_input_path = f"s3://{sess.default_bucket()}/{s3_prefix}/train"
print(training_input_path)
train_dataset.save_to_disk(training_input_path, fs=s3)

# save test_dataset to s3
eval_input_path = f"s3://{sess.default_bucket()}/{s3_prefix}/eval"
eval_dataset.save_to_disk(eval_input_path, fs=s3)

s3://sagemaker-us-west-2-570106654206/samples/datasets/squad-8million/train


### SageMaker Training Job

To create a SageMaker training job, we use a `HuggingFace Estimator`. Using the estimator, you can define which script should SageMaker use through `entry_point`, which `instance_type` to use for training, which `hyperparameters` to pass, and so on.

When a SageMaker training job starts, SageMaker takes care of starting and managing all the required machine learning instances, picks up the `HuggingFace Deep Learning Container`, uploads your training script, and downloads the data from `sagemaker_session_bucket` into the container at `/opt/ml/input/data`.

In [9]:
from sagemaker.huggingface import HuggingFace

subnets = ["subnet-031c4b8933ae6f4fb"]  # Should be same as Subnet used for FSx. Example: subnet-0f9XXXX
security_group_ids = ["sg-095b065a4a6bb7136"]  # Should be same as Security group used for FSx. sg-03ZZZZZZ
job_name = "smdataparallel-deberta-fsx-trial-16"  # This job name is used as prefix to the sagemaker training job. Makes it easy for your look for your training job in SageMaker Training job console.
file_system_id = "fs-01cd8aadfdbf45e8c"  # FSx file system ID with your training dataset. Example: 'fs-0bYYYYYY'

# hyperparameters, which are passed into the training script
hyperparameters={
    'epochs': 20,                                    
    'train_batch_size': 16,                         
    'eval_batch_size': 16,                          
    'learning_rate': 3e-5*8
}

# refer https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-training-containers to get the right uri's based on region
#image_uri = '763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.9.1-transformers4.12.3-gpu-py38-cu111-ubuntu20.04'
image_uri = '570106654206.dkr.ecr.us-west-2.amazonaws.com/pt-hf-110

# configuration for running training on smdistributed Data Parallel
# this is the only line of code change required to leverage SageMaker Distributed Data Parallel
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'qa_deberta_huggingface_trainer.py',       
    source_dir           = './deberta_script',       
    instance_type        = 'ml.p4d.24xlarge',   
    instance_count       = 1, 
    role                 = role,             
    py_version           = 'py38',  
    sagemaker_session    = sess,
    image_uri            = image_uri,
    hyperparameters      = hyperparameters,   
    subnets              = subnets,
    security_group_ids   = security_group_ids,
    debugger_hook_config = False,
    distribution         = distribution,
    max_retry_attempts   = 30
)

In [10]:
# Configure FSx Input for your SageMaker Training job

from sagemaker.inputs import FileSystemInput

# YOUR_MOUNT_PATH_FOR_TRAINING_DATA # Example: '/fsx/squad...''''
file_system_directory_path_train = "/nx36zbmv/squad/squad/train"
file_system_directory_path_eval = "/nx36zbmv/squad/squad/eval"
#file_system_directory_path_train = "/uzr6zbmv/squad/squad/train"
#file_system_directory_path_eval = "/uzr6zbmv/squad/squad/eval"
file_system_access_mode = "rw"
file_system_type = "FSxLustre"
train_fs = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path_train,
    file_system_access_mode=file_system_access_mode,
)
eval_fs = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path_eval,
    file_system_access_mode=file_system_access_mode,
)
data_channels = {"train": train_fs, "eval": eval_fs}

In [11]:
# starting the train job with data in FSx as input
huggingface_estimator.fit(inputs=data_channels,job_name=job_name,wait=False)